In [ ]:
#train_pure = '/kaggle/input/break224/break224/gcn_224/Train_gcn/*'
#val_pure =  '/kaggle/input/break224/break224/gcn_224/Val_gcn/*'
train_pure = 'gcn_224/Train_gcn/*'
val_pure =  'gcn_224/Val_gcn/*'

In [ ]:
import glob
train_files = glob.glob(train_pure)
val_files = glob.glob(val_pure)
len(train_files)#6229

In [ ]:
len(val_files)

In [ ]:
#!pip install barbar
#!pip install swin-transformer-pytorch

In [ ]:
from torchvision import transforms
#import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import albumentations
import albumentations.pytorch 
import cv2
import torch.nn as nn
import copy
from barbar import Bar
import io
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
torch.cuda.device_count()

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark=True
    torch.backends.cudnn.deterministic=False  

In [ ]:
class My_data(Dataset):
    def __init__(self, data, transforms=None):
        self.image_list = data
        self.data_len = len(self.image_list)
        self.transforms = transforms
        self.eicls = ["A", "F", "TA", "PT", "DC", "LC", "MC", "PC"]
        
    def __getitem__(self, index):
        current_image_path = self.image_list[index]
        im_as_im = cv2.imread(current_image_path)
        im_as_im = cv2.cvtColor(im_as_im, cv2.COLOR_BGR2RGB)

        # Perform label encoding for multi-label classification
        parts = current_image_path.split('_')[-1].split('-')
        if parts[2]=="13412":
            labels =[0,0,0,0,1,1,0,0]
        else:
            labels = [int(label == parts[0]) for label in self.eicls]       
        labels = torch.tensor(labels)

        if self.transforms is not None:
            augmented = self.transforms(image=im_as_im)
            im_as_im = augmented['image']

        return (im_as_im, labels)

    def __len__(self):
        return self.data_len

In [ ]:
transform = {
    'train': albumentations.Compose([
    albumentations.Resize(224, 224),     
    albumentations.OneOf([
                          albumentations.HorizontalFlip(),
                          albumentations.RandomRotate90(),
                          albumentations.VerticalFlip()            
    ], p=0.75),
    albumentations.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p=1),
    albumentations.pytorch.transforms.ToTensorV2()]),
    
    'valid': albumentations.Compose([
    albumentations.Resize(224, 224),     
    albumentations.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p=1),
    albumentations.pytorch.transforms.ToTensorV2()]),
    
}

In [ ]:
train=My_data(train_files,transforms=transform['train'])
valid=My_data(val_files,transforms=transform['valid'])

In [ ]:
a=torch.tensor([0,0,0,0,0,0,0,0])
for _,label in train:
    a=a+label
print(a)     


In [ ]:
b=torch.tensor([0,0,0,0,0,0,0,0])
for _,label in valid:
    b=b+label
print(b)     


In [ ]:
import torch

class_samples = [367, 803, 456, 370, 2763, 492, 629, 449]  # Number of samples in each class

total_samples = sum(class_samples)
class_weights = [total_samples / (s + 1e-8) for s in class_samples]
class_weights_sum = sum(class_weights)
class_weights_normalized = [w / class_weights_sum for w in class_weights]
class_weights_normalized = torch.tensor(class_weights_normalized)

print(class_weights_normalized)

In [ ]:
len(valid)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset=train, batch_size=4,shuffle=True,num_workers=2,
                                              pin_memory=True,prefetch_factor=2)
valid_dataloader=  torch.utils.data.DataLoader(dataset=valid,batch_size=4,shuffle=False,num_workers=2,
                                               pin_memory=True ,prefetch_factor=2)  

In [ ]:
for i in valid_dataloader:
    print(i)
    break    

In [ ]:
#!pip install timm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from timm.models import create_model
from timm.data import create_transform
from sklearn.metrics import accuracy_score

# Define device
# Define Swin Transformer v2 model
model_name = 'swin_base_patch4_window7_224'
num_classes = 8
model = create_model(
    model_name=model_name,
    pretrained=True,
    num_classes=num_classes,
    drop_rate=0.5,
    drop_path_rate=0.2,
    checkpoint_path=None
)


for param in model.parameters():
    param.requires_grad = True
"""for param in model.head.parameters():
    param.requires_grad = True
for param in model.norm.parameters():
    param.requires_grad = True  """ 
#model=model.to(device)

In [ ]:
model

import torch
for param in model.layers.parameters():
    param.requires_grad = False
for param in model.layers[3].parameters():
    param.requires_grad = True    

# Iterate over the parameters and check requires_grad
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter '{name}' requires grad.")
    else:
        print(f"Parameter '{name}' does not require grad.")


In [1]:
import torch
import timm
model = timm.create_model(
    'swinv2_small_window8_256.ms_in1k',
    pretrained=True,
    features_only=False,
    num_classes = 8,drop_rate=0.5,
    drop_path_rate=0.2, 
    checkpoint_path=None 
)

/home/kb/anaconda3/envs/pytorch-gpu/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
model

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1): Identi

In [ ]:
#model.layers[0].blocks[0].attn

In [ ]:
#block1

In [ ]:
#block1.drop_path2

In [ ]:
#attention1

In [3]:
import torch

block0 = model.layers[0].blocks [0]
block1 = model.layers[0].blocks [1]
attention0=block0.attn
attention1=block1.attn

class ParallelAttention (torch.nn.Module):
    def __init__ (self, attention0, attention1):
        super ().__init__ ()
        self.attention0 = attention0
        self.attention1 = attention1
        self.layernorm1 = block1.norm1 # copy the layernorm_before module from block0
        self.drop_path1 = block1.drop_path1 # copy the drop_path module from block0
        self.mlp = block1.mlp # copy the intermediate module from block0
        self.norm2 = block1.norm2 # copy the output module from block0
        self.drop_path2 = block1.drop_path2
    

def forward (self, x):
    x0 = self.attention0 (x) # apply attention module of block0
    x1 = self.attention1 (x) # apply attention module of block1
    x = torch.cat ([x0, x1], dim= -1) # concatenate along the last dimension
    x = self.layernorm1 (x) # apply layernorm_before module
    x = self.drop_path1 (x) # apply drop_path module
    x = self.mlp (x) # apply intermediate module
    x = self.drop_path2 (x) # apply output module
    x=  self.norm2 (x)
    

In [4]:
block0 = model.layers[0].blocks [0]
block1 = model.layers[0].blocks [1]
attention0=block0.attn
attention1=block1.attn
new_block0=ParallelAttention(attention0,attention1)
model.layers[0].blocks=new_block0

In [5]:
block0 = model.layers[1].blocks [0]
block1 = model.layers[1].blocks [1]
attention0=block0.attn
attention1=block1.attn
new_block1=ParallelAttention(attention0,attention1)
model.layers[1].blocks=new_block1

In [6]:
block0 = model.layers[2].blocks [0]
block1 = model.layers[2].blocks [1]
attention0=block0.attn
attention1=block1.attn
new_block2=ParallelAttention(attention0,attention1)
model.layers[2].blocks[0]=new_block2

In [7]:
block0 = model.layers[2].blocks [2]
block1 = model.layers[2].blocks [3]
attention0=block0.attn
attention1=block1.attn
new_block3=ParallelAttention(attention0,attention1)
model.layers[2].blocks[1]=new_block3

In [8]:
block0 = model.layers[2].blocks [4]
block1 = model.layers[2].blocks [5]
attention0=block0.attn
attention1=block1.attn
new_block4=ParallelAttention(attention0,attention1)
model.layers[2].blocks[2]=new_block4

In [9]:
block0 = model.layers[2].blocks [6]
block1 = model.layers[2].blocks [7]
attention0=block0.attn
attention1=block1.attn
new_block5=ParallelAttention(attention0,attention1)
model.layers[2].blocks[3]=new_block5

In [10]:
block0 = model.layers[2].blocks [8]
block1 = model.layers[2].blocks [9]
attention0=block0.attn
attention1=block1.attn
new_block6=ParallelAttention(attention0,attention1)
model.layers[2].blocks[4]=new_block6

In [11]:
block0 = model.layers[2].blocks [10]
block1 = model.layers[2].blocks [11]
attention0=block0.attn
attention1=block1.attn
new_block7=ParallelAttention(attention0,attention1)
model.layers[2].blocks[5]=new_block7

In [12]:
block0 = model.layers[2].blocks [12]
block1 = model.layers[2].blocks [13]
attention0=block0.attn
attention1=block1.attn
new_block8=ParallelAttention(attention0,attention1)
model.layers[2].blocks[6]=new_block8

In [13]:
block0 = model.layers[2].blocks [14]
block1 = model.layers[2].blocks [15]
attention0=block0.attn
attention1=block1.attn
new_block9=ParallelAttention(attention0,attention1)
model.layers[2].blocks[7]=new_block9

In [14]:
block0 = model.layers[2].blocks [16]
block1 = model.layers[2].blocks [17]
attention0=block0.attn
attention1=block1.attn
new_block10=ParallelAttention(attention0,attention1)
model.layers[2].blocks[7]=new_block10

In [15]:
block0 = model.layers[3].blocks [0]
block1 = model.layers[3].blocks [1]
attention0=block0.attn
attention1=block1.attn
new_block11=ParallelAttention(attention0,attention1)
model.layers[3].blocks=new_block11

In [16]:
for i in range(17, 7, -1):
    del model.layers[2].blocks[i]

In [17]:
model

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ParallelAttention(
        (attention0): WindowAttention(
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
          (qkv): Linear(in_features=96, out_features=288, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (softmax): Softmax(dim=-1)
        )
        (attention1): WindowAttention(
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
          

In [ ]:
#train in this case for 10 epochs lr constraint on layer[3]
for param in model.parameters():
    param.requires_grad = False
for param in model.layers[3].parameters():
    param.requires_grad = True
for param in model.norm.parameters():
    param.requires_grad = True
for param in model.head.parameters():
    param.requires_grad = True

In [ ]:
import time
epochs=50
for epoch in range(epochs):
    time.sleep(1)
    epoch=epoch+1
    if epoch<=9:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True
            if epoch== 9:
                for name, param in model.named_parameters():
                    if param.requires_grad:
                        print(name) 
                
    elif  10<epoch<=19 :
        for param in model.layers[2].parameters():
            param.requires_grad = True
    elif  20<epoch<=29:
        for param in model.layers[1].parameters():
            param.requires_grad = True
    elif  30<epoch<=39:
        for param in model.layers[0].parameters():
            param.requires_grad = True
    else: 
        for param in model.parameters():
            param.requires_grad = True




In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
for param in model.patch_embed.parameters():
    param.requires_grad = True    
for param in model.layers[0].parameters():
    param.requires_grad = True
for param in model.layers[1].parameters():
    param.requires_grad = True   
for param in model.layers[2].parameters():
    param.requires_grad = True  
for param in model.layers[3].parameters():
    param.requires_grad = True
for param in model.norm.parameters():
    param.requires_grad = True
for param in model.head.parameters():
    param.requires_grad = True


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

In [ ]:
x=model.layers[3].blocks[0].attn
y=model.layers[3].blocks[1].attn

In [ ]:
z=model.layers[3].blocks[1].mlp

In [ ]:
z

In [ ]:
x

In [ ]:
y

In [ ]:
import torch.nn as nn
class ConcatenatedModule(nn.Module):
    def __init__(self, layer1, layer2,layer3):
        super(ConcatenatedModule, self).__init__()
        self.layer1 = layer1
        self.layer2 = layer2
        self.layer3 = layer3

    def forward(self, x):
        output1 = self.layer1(x)
        output2 = self.layer2(x)
        output3 = torch.cat((output1, output2), dim=1)
        
        output= self.layer3(output3)
        return output


In [ ]:
concatenated_module = ConcatenatedModule(x,y,z)


In [ ]:
concatenated_module


In [ ]:
y=model.stages[-1].blocks[0]
y

In [ ]:
import torch
import torch.nn as nn

class FocalLossWithClassWeights(nn.Module):
    def __init__(self, class_weights, alpha=1, gamma=2):
        super(FocalLossWithClassWeights, self).__init__()
        self.class_weights = class_weights
        self.alpha = alpha
        self.gamma = gamma
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, target):
        class_weights = self.class_weights.to(target.device)
        weighted_logits = class_weights * input
        probs = self.sigmoid(weighted_logits)

        loss = -(self.alpha * torch.pow(1 - probs, self.gamma) * target * torch.log(probs + 1e-8)
                 + (1 - target) * torch.log(1 - probs + 1e-8))

        return loss.mean()



for param in model.parameters():
    param.requires_grad =True
    #print(param.requires_grad)


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
#model = nn.DataParallel(model, device_ids = [0, 1])
model = model.to(device)
class_weights_normalized=class_weights_normalized.to(device)
#criterion = torch.nn.BCEWithLogitsLoss(weight=class_weights_normalized)
criterion = FocalLossWithClassWeights(class_weights_normalized)
#criterion= torch.nn.MultiLabelSoftMarginLoss()
#criterion =torch.nn.CrossEntropyLoss()

#optimizer = optims.Adahessian(model.parameters(),lr=0.001,hessian_power=0.5,weight_decay=0.1)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
best_model_wts = model.state_dict()
best_optimizer_state =optimizer.state_dict()
best_acc = 0.0

In [ ]:
def fit(model, dataloader, optimizer,scheduler, criterion):
    #print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    accum_iter = 4  
    
    for i, (inputs, labels) in enumerate(Bar(dataloader)):
            inputs = inputs.to(device)           
            labels = labels.float().to(device)
            optimizer.zero_grad()
            #model.zero_grad(set_to_none=True)
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            thresholds = [0.5, 0.5, 0.5,0.5,0.5,0.5,0.5,0.5]
            # Compute loss
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()* inputs.size(0)
           # _ , preds = torch.max(outputs.data, 1)
            # Apply sigmoid activation to obtain probabilities
            #preds = (outputs > 0.5).float()
            probs = torch.sigmoid(outputs)
            preds = torch.zeros_like(probs)
            
            # Set predicted labels based on the threshold
            for i, threshold in enumerate(thresholds):
                preds[:, i] = (probs[:, i] >= threshold).float()
            train_running_correct += (preds == labels).all(dim=1).float().sum()
            # Backpropagate the gradients
            loss /= accum_iter
            loss.backward() 
            
                       
            if ((i + 1) % accum_iter == 0) :
                optimizer.step()
                optimizer.zero_grad()
                    
            
    scheduler.step()
            
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(dataloader.dataset)    
    return train_loss, train_accuracy

In [ ]:
def validate(model, dataloader, optimizer, criterion):
    #print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.float()
            labels = labels.to(device)
            outputs = model(inputs)
            thresholds = [0.5, 0.5, 0.5,0.5,0.5,0.5,0.5,0.5]
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item()*inputs.size(0)
            #_, preds = torch.max(outputs.data, 1)
            #preds = (outputs > 0.5).float()
            probs = torch.sigmoid(outputs)
            preds = torch.zeros_like(probs)
            # Set predicted labels based on the threshold
            for i, threshold in enumerate(thresholds):
                preds[:, i] = (probs[:, i] >= threshold).float()
            val_running_correct += (preds == labels).all(dim=1).float().sum()
        
    val_loss = val_running_loss/len(dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(dataloader.dataset)        
    return val_loss, val_accuracy

In [ ]:
import time as time
history=[]
best_model_wts = copy.deepcopy(model.state_dict())
#best_optimizer_state = copy.deepcopy(optimizer.state_dict())
best_acc = 0.0
epochs=50

for epoch in range(epochs):
    epoch_start = time.time()
    print('Epoch-{0}/{1} lr: {2}'.format(epoch+1,epochs ,optimizer.param_groups[0]['lr']))
    if epoch<=9:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True
                       
    elif  10<epoch<=19 :
        for param in model.layers[2].parameters():
            param.requires_grad = True
    elif  20<epoch<=29:
        for param in model.layers[1].parameters():
            param.requires_grad = True
    elif  30<epoch<=39:
        for param in model.layers[0].parameters():
            param.requires_grad = True
    else: 
        for param in model.parameters():
            param.requires_grad = True
    
    train_epoch_loss, train_epoch_accuracy = fit(model,train_dataloader,optimizer,scheduler,criterion)
    val_epoch_loss, val_epoch_accuracy = validate(model,valid_dataloader,optimizer,criterion)
    
    epoch_end = time.time()
    history.append([epoch+1,train_epoch_loss, train_epoch_accuracy, val_epoch_loss, val_epoch_accuracy,(epoch_end-epoch_start)])
    print(f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f},Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_accuracy:.2f},time : {epoch_end-epoch_start:.2f}")
    torch.save({'history':history},'Master_his.pth')
    if val_epoch_accuracy > best_acc:
        best_acc = val_epoch_accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
       
        best_epoch=epoch
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': best_model_wts,
            'loss': criterion,
            'history':history,
            'best_epoch': best_epoch+1,          
    
            }, 'Master.pth')    
  

In [ ]:
import torch
import torch.nn as nn
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, class_weights=None):
        super(FocalLoss, self).__init__() 
        self.gamma = gamma 
        self.class_weights = class_weights
        
    def forward(self, logits, labels):
        probs = torch.sigmoid(logits) 
        ce_loss = nn.BCELoss()(probs, labels) 
        weight = torch.pow(1 - probs, self.gamma) 
        if self.class_weights is not None: 
            weight = weight * self.class_weights 
            loss = ce_loss * weight 
        return loss


In [ ]:
import time
epochs=50
for epoch in range(epochs):
    time.sleep(1)
    epoch=epoch+1
    if epoch<=9:
        for param in model.parameters():
            param.requires_grad = False        
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True       
    
    else: 
        for param in model.parameters():
            param.requires_grad = True



In [ ]:
import time as time
history=[]
best_model_wts = copy.deepcopy(model.state_dict())
#best_optimizer_state = copy.deepcopy(optimizer.state_dict())
best_acc = 0.0
epochs=50

for epoch in range(epochs):
    epoch_start = time.time()
    print('Epoch-{0}/{1} lr: {2}'.format(epoch+1,epochs ,optimizer.param_groups[0]['lr']))
    if epoch <= 9:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    elif 10 <= epoch <= 19:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[2].parameters():
            param.requires_grad = True
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    elif 20 <= epoch <= 29:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[1].parameters():
            param.requires_grad = True
        for param in model.layers[2].parameters():
            param.requires_grad = True
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    elif 30 <= epoch <= 39:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layers[0].parameters():
            param.requires_grad = True
        for param in model.layers[1].parameters():
            param.requires_grad = True
        for param in model.layers[2].parameters():
            param.requires_grad = True
        for param in model.layers[3].parameters():
            param.requires_grad = True
        for param in model.norm.parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True
    else:
        for param in model.parameters():
            param.requires_grad = True

        
    train_epoch_loss, train_epoch_accuracy = fit(model,train_dataloader,optimizer,scheduler,criterion)
    val_epoch_loss, val_epoch_accuracy = validate(model,valid_dataloader,optimizer,criterion)
    
    epoch_end = time.time()
    history.append([epoch+1,train_epoch_loss, train_epoch_accuracy, val_epoch_loss, val_epoch_accuracy,(epoch_end-epoch_start)])
    print(f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f},Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_accuracy:.2f},time : {epoch_end-epoch_start:.2f}")
    torch.save({'history':history},'Master_his.pth')
    if val_epoch_accuracy > best_acc:
        best_acc = val_epoch_accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
       
        best_epoch=epoch
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': best_model_wts,
            'loss': criterion,
            'history':history,
            'best_epoch': best_epoch+1,          
    
            }, 'Master.pth')    
  